## Write A Data Science Blog Post
##### by Markus Müller


Content:
1. Gather
2. Assess
3. Clean



Three Questions:
1. Availability and prive development
2. which neighborhoods are most expensive
3. Which amenities can be expected at a given price?
4. What feature predicts price 

Data: Boston Airbnb Open Data from <a href='https://www.kaggle.com/airbnb/boston'>Kaggle</a>

Structure:
- listings: full descriptions and average review score
- reviews: unique id for each reviewer and detailed comments
- calendar: listing id and the price and availability for that day

In [1]:
# imports
import datetime
import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import mplleaflet

import plotly.express as px
import plotly.io as pio # export plotly visualizations to HTML


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# set seaborn style to get uniform visualizations

#set base color
b_color = sns.color_palette()[0]
# set seaborn style for uniform 
sns.set_style('whitegrid')

### Gather

In [3]:
# create folder
folder_name = 'Data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [4]:
calendar = pd.read_csv('Data/calendar.csv')
listings = pd.read_csv('Data/listings.csv')
reviews = pd.read_csv('Data/reviews.csv')

### Assess

In [ ]:
calendar

In [ ]:
calendar.info()

In [ ]:
calendar.isnull().sum()

In [ ]:
calendar[calendar['price'].notna()]

In [ ]:
# availabe (t = True / f = False)
calendar.query('available == "f"')

In [ ]:
listings

In [ ]:
listings.info()

In [ ]:
# used this cell to visually assess different columns
listings[['id', 'price', 'weekly_price', 'monthly_price', 'cleaning_fee']]

In [ ]:
# used this cell to visually assess different columns
listings[['id', 'host_since', 'first_review', 'last_review', 'neighbourhood', 'neighbourhood_cleansed', 'property_type']]

In [ ]:
# listings['price'].describe

In [ ]:
# listings['price'].hist(bins=100);

In [ ]:
# check distribution of reviews
listings['review_scores_rating'].hist(bins=100);

In [ ]:
# check for outliers
#listings.query('price_clean <= 1400.0')

In [ ]:
listings.isnull().sum()

In [ ]:
reviews

In [ ]:
reviews.info()

#### Result Data Assessment

##### calendar
- `price` is an object 
- `date` should be a DateTimeObject

##### lisitings
- `price` in an object 
- `price` has outliers
- `host_since` should be a DateTimeObject

##### reviews
- `date` should be a DateTimeObject

### Data Cleaning

In [5]:
# copy DataFrames
calendar_clean = calendar.copy()
listings_clean = listings.copy()
reviews_clean = reviews.copy()

##### clean price coloumns

In [6]:
# function to clean and transform 
def clean_price(df, p_column):
    df['price_clean'] = df[p_column].str.replace('[$,]', '')
    df['price_clean'] = df['price_clean'].astype(float)
    df.drop(columns=p_column, inplace=True)

In [7]:
# apply function
clean_price(calendar_clean, 'price')
clean_price(listings_clean, 'price')

In [8]:
# check values
calendar_clean['price_clean'].value_counts()

150.0     20752
65.0      17518
75.0      16249
100.0     15499
200.0     14326
          ...  
2415.0        1
1148.0        1
1149.0        1
2046.0        1
1617.0        1
Name: price_clean, Length: 1246, dtype: int64

In [9]:
# check values
listings_clean['price_clean'].value_counts()

150.0    144
200.0    117
100.0    101
250.0     95
75.0      94
        ... 
241.0      1
211.0      1
132.0      1
599.0      1
999.0      1
Name: price_clean, Length: 324, dtype: int64

##### remove outliers in price columns

In [10]:
listings_clean = listings_clean.query('price_clean <= 1400.0')

In [ ]:
#calendar_clean.query('price_clean >= 4000')['price_clean'].unique()

##### transforme date to DateTimeObject and extract day of week

In [11]:
calendar_clean['date'] = pd.to_datetime(calendar_clean['date'], format='%Y-%m-%d')
listings_clean['host_since'] = pd.to_datetime(listings_clean['host_since'], format='%Y-%m-%d')
reviews_clean['date'] = pd.to_datetime(reviews_clean['date'], format='%Y-%m-%d')

In [12]:
# get day of week from DateTimeObject
calendar_clean['day_of_week'] = calendar_clean['date'].dt.strftime('%a')

In [13]:
calendar_clean.head(5)

,listing_id,date,available,price_clean,day_of_week
0,12147973,2017-09-05,f,NaN,Tue
1,12147973,2017-09-04,f,NaN,Mon
2,12147973,2017-09-03,f,NaN,Sun
3,12147973,2017-09-02,f,NaN,Sat
4,12147973,2017-09-01,f,NaN,Fri


### Visualizations

In [ ]:
def creat_price_cat(row):
    if row['price_clean'] >= 400:
        return 1
    elif row['price_clean'] >= 200:
        return 2
    elif row['price_clean'] >= 100:
        return 3
    else:
        return 4

In [ ]:
listings_clean['price_cat'] = listings_clean.apply(creat_price_cat, axis=1)

In [ ]:
listings_clean[['price_clean', 'price_cat']]

In [ ]:
plt.hist(listings_clean['price_clean'], bins=150)
plt.xlim(0,1500);

I assume that the price in the listings table refelct the price from the day that the data was scraped, whereas the price in the calendar DataFrame shows the price of an individuall listing over the timeperiod

In [ ]:
price_cat_1 = listings_clean.query('price_cat == 1')[['id', 'longitude', 'latitude', 'price_clean', 'price_cat']]
price_cat_2 = listings_clean.query('price_cat == 2')[['id', 'longitude', 'latitude', 'price_clean', 'price_cat']]
price_cat_3 = listings_clean.query('price_cat == 3')[['id', 'longitude', 'latitude', 'price_clean', 'price_cat']]
price_cat_4 = listings_clean.query('price_cat == 4')[['id', 'longitude', 'latitude', 'price_clean', 'price_cat']]

In [ ]:
plt.subplots(figsize=(8,8))
plt.scatter(price_cat_1['longitude'], price_cat_1['latitude'], s=3, alpha=1, label = '>=400', color='red')
plt.scatter(price_cat_2['longitude'], price_cat_2['latitude'], s=3, alpha=0.2, label = '>=200', color='blue')
plt.scatter(price_cat_3['longitude'], price_cat_3['latitude'], s=3, alpha=0.2, label = '>=100', color='blue')
plt.scatter(price_cat_4['longitude'], price_cat_4['latitude'], s=3, alpha=0.2, label = '<100', color='blue')
plt.legend();

In [ ]:
# knn to find clusters compare price between them

### Question X: Availability

In [ ]:
# create a DataFrame for each instance
available = calendar_clean.query('available == "t"')

In [ ]:
available_price = calendar_clean.query('available =="t"').groupby('date')['price_clean'].mean()

In [ ]:
# grouoby date to see the availability of each day
available_plot = available.groupby('date').count()

In [ ]:
fig, ax1 = plt.subplots(figsize = (10,6))

color = 'tab:green'
ax1.set_xlabel('date')
ax1.set_ylabel('available', color=color)
ax1.plot(available_plot.index, available_plot['listing_id'], label = 'available', color = color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('price', color=color)  # we already handled the x-label with ax1
ax2.plot(available_price.index, available_price.values, label = 'price', color = color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Availability')
fig.legend(loc='upper right', bbox_to_anchor=(0.92, 0.96));
# https://matplotlib.org/gallery/api/two_scales.html

- there seems to be some seasonality in the price, because it has almost the same pattern 

In [ ]:
# transform to cat to polt days of week in order
available['day_of_week'] = pd.Categorical(available['day_of_week'], 
                                          categories=['Mon','Tue','Wed','Thu','Fri','Sat', 'Sun'], 
                                          ordered=True)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.barplot(x='day_of_week', y='price_clean', data=available, color=b_color, ci= None)
plt.title('Averag prices on weekdays')
plt.ylabel('Average price')
plt.xlabel('Day of Week')
plt.ylim(150,215);

there is a slightly higher price on Fridays and Saturdays

### Question X: Where to stay?

In [ ]:
plt.subplots(figsize=(8,8))
plt.scatter(listings_clean['longitude'], listings_clean['latitude'], s=3, alpha=0.5)
#mplleaflet.display()

In [ ]:
token = 'pk.eyJ1IjoibWFya2luZ2VyIiwiYSI6ImNrYXAzc2poOTBqc2gydXFudWdxeDAzb3kifQ._su7Ef8qqFBHuXa3tR1w9Q'
fig = px.scatter_mapbox(listings_clean, lat='latitude', lon="longitude", 
                        hover_data=['price_clean', 'neighbourhood_cleansed'],
                        color='price_clean', zoom=10, height=300,
                        #color_continuous_scale=px.colors.cyclical.IceFire)
                       )
fig.update_layout(mapbox_style='light', mapbox_accesstoken=token)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.show()
# https://plotly.com/python/mapbox-layers/
# saves the output as HTML to use it in the Blog article
pio.write_html(fig, file='index.html', auto_open=True)
# https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b

### Question X: Neighberhoods

In [ ]:
neighbourhoods = listings_clean[['id', 'neighbourhood', 'neighbourhood_cleansed', 'price_clean']]

In [ ]:
neighbourhoods['neighbourhood_cleansed'].nunique()

In [ ]:
neighbourhoods['neighbourhood_cleansed'].value_counts()

In [ ]:
neighbourhoods.groupby('neighbourhood_cleansed')['price_clean'].mean().sort_values(ascending = False)

In [ ]:
neighbourhoods.groupby('neighbourhood_cleansed')['price_clean'].max().sort_values(ascending = False).head(5)

In [ ]:
neighbourhoods['price_clean'].mean()

In [ ]:
neighbourhoods['diff_to_mean'] = neighbourhoods['price_clean'] - neighbourhoods['price_clean'].mean()

In [ ]:
neighbourhoods

In [ ]:
fig = plt.subplots(figsize=(6,10))
sns.barplot(y='neighbourhood_cleansed', x='diff_to_mean', data=neighbourhoods, 
            order=neighberhoods.groupby('neighbourhood_cleansed')['diff_to_mean'].mean().sort_values().index,
            ci= None, color=b_color)
#plt.xticks(rotation=90)
plt.title('Average price difference to mean price of Boston AirBnBs')
plt.ylabel('Neighbourhoods in Boston')
plt.xlabel('Average price difference')
plt.tight_layout()

### Question X: Which amenities can be expected at a given price?

In [ ]:
amenities = listings_clean[['id', 'price_clean', 'amenities']]

In [ ]:
amenities['amenities'] = amenities['amenities'].str.replace("[{}]", "")

In [ ]:
amenity_dummy = amenities['amenities'].str.get_dummies(sep = ",")

In [ ]:
amenity_dummy.drop(columns=['"translation missing: en.hosting_amenity_49"', '"translation missing: en.hosting_amenity_50"'], inplace=True)

In [ ]:
amenity_dummy.columns = amenity_dummy.columns.str.replace('"','')

In [ ]:
amenity_dummy.head(2)

### Question X: What contributes to a high price

In [14]:
# prices higher than 600 will be romoved fot this question
listings_clean = listings_clean.query('price_clean <= 600')

In [15]:
#create a DataFrame which contains possible variables influencing price 
model = listings_clean[['id', 'bedrooms', 'beds', 'bed_type', 'room_type','bathrooms', 'property_type', 
                        'neighbourhood_cleansed', 'cancellation_policy', 'number_of_reviews',
                        'review_scores_rating', 'reviews_per_month', 'host_is_superhost', 'amenities',
                        'cancellation_policy', 'guests_included', 'square_feet', 'price_clean']]

In [16]:
model.isnull().sum()

id                           0
bedrooms                    10
beds                         9
bed_type                     0
room_type                    0
bathrooms                   14
property_type                3
neighbourhood_cleansed       0
cancellation_policy          0
number_of_reviews            0
review_scores_rating       798
reviews_per_month          742
host_is_superhost            0
amenities                    0
cancellation_policy          0
guests_included              0
square_feet               3492
price_clean                  0
dtype: int64

In [22]:
model['bedrooms'].value_counts()

1.0    2367
2.0     684
0.0     287
3.0     147
4.0      39
5.0      13
Name: bedrooms, dtype: int64

In [25]:
model['beds'].corr(model['bedrooms'])

0.6975409285801463

In [29]:
model['bathrooms'].mean()#.value_counts()

1.2135578828191338

In [31]:
model['property_type'].value_counts()

Apartment          2586
House               554
Condominium         228
Townhouse            53
Bed & Breakfast      41
Loft                 39
Other                17
Boat                 12
Villa                 6
Entire Floor          4
Dorm                  2
Guesthouse            1
Camper/RV             1
Name: property_type, dtype: int64

In [36]:
model['review_scores_rating'].median()

(94.0, 91.88141142233539)

In [40]:
model['reviews_per_month'].value_counts()

1.00    114
2.00     53
0.08     28
3.00     26
0.09     26
       ... 
6.96      1
5.83      1
6.13      1
4.95      1
4.89      1
Name: reviews_per_month, Length: 644, dtype: int64

How to handle missing values:
- `square_feet` gets droped, because it has to many missing values
- `bedrooms`: since the mean is 1.24 and mode is 1, all missing values will get filled with 1
- `beds`: gets droped -> correlates with bedrooms
- `bathrooms`: since the mean is 1.21 and the mode is 1, all missing values will get filled with 1
- `property_type`: the three missing ones will be added to other
- `review_scores_rating`: because there is no pattern behind the missing value I will use the random sample method
- `reviews_per_month`: random sample method

more methods: https://towardsdatascience.com/8-clutch-ways-to-impute-missing-data-690481c6cb2b

##### fill missing values

In [89]:
# drop square_feet and beds
model.drop(columns=['square_feet', 'beds'], inplace=True)

In [92]:
# bedrooms and bathrooms fill missing values with 1
model['bedrooms'].fillna(value=1, inplace=True)
model['bathrooms'].fillna(value=1, inplace=True)

In [93]:
# change property_type 
model['property_type'].fillna(value='Other', inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [81]:
#  https://stackoverflow.com/questions/36413314/filling-missing-data-by-random-choosing-from-non-missing-values-in-pandas-datafr
def fill_with_random(df2, column):
    '''Fill `df2`'s column with name `column` with random data based on non-NaN data from `column`'''
    df = df2.copy()
    df[column] = df[column].apply(lambda x: np.random.choice(df[column].dropna().values) if np.isnan(x) else x)
    return df

In [85]:
model = fill_with_random(model, 'review_scores_rating')
model = fill_with_random(model, 'reviews_per_month')

In [ ]:
#colums which could be ralated to the price 
price_corr = listings_clean[['price_clean', 'bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month']]

In [ ]:
# review_score_rating is the combinatiom of the other reveiw_scores
# listings_clean[['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',
#                'review_scores_communication', 'review_scores_location', 'review_scores_value']]

In [ ]:
sns.heatmap(price_corr.corr(), annot=True);

There are some medium correlations for bathrooms, bedrooms, bad and very small correlations for review and review per month. Intersting is that a high reveiw doesn't seem to affact price.

In [ ]:
score = listings_clean.groupby('review_scores_rating')['price_clean'].mean().index
mean_price = listings_clean.groupby('review_scores_rating')['price_clean'].mean().values

In [ ]:
plt.bar(x=score, height=mean_price)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='host_is_superhost', y='price_clean', data=listings_clean)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='property_type', y='price_clean', data=listings_clean, 
            order=listings_clean.groupby('property_type')['price_clean'].mean().sort_values(ascending=False).index)
plt.xticks(rotation=50)
plt.tight_layout();

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='bed_type', y='price_clean', data=listings_clean)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='room_type', y='price_clean', data=listings_clean)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='cancellation_policy', y='price_clean', data=listings_clean)

In [ ]:
fig = plt.subplots(figsize=(8,6))
sns.boxplot(x='guests_included', y='price_clean', data=listings_clean)

#### Linear Regression to see what impacts price
What features should the model includ?
- `neighbourhood_cleansed` showed varianze between different neighbourhoods.
- `bathrooms` as well as `bedromms` had a medium correlation
- `review_scores_rating` hadn't a high correlation and the visual assesment didn't show an obvious pattern 
- `square_feet` would be interesting, but there are to many missing values
- `guests_included`

- property type

To do a linear regression we need to make some transformations:
- `price` is right skewed. To solve this we can transform the price, but then we will loose the interpretability so in this case price won't be transformed.
- `review_scores_rating` is left skewed, wo have to log the review score
- categorical columns like: `neighbourhood_cleansed`, `bathrooms`, `cancellation_policy`, `bedrooms`, `property_type` need to be transformed to dummy variables
- `property_type`: unpopular tyoes will be summarized as other
- the boxplots revealed more outliers in price so I will remove prices higher than $600
- `guests_included`: combine 7 and more into one category 

In [ ]:
listings_clean['price_clean'].hist(bins=100);

In [ ]:
listings_clean['review_scores_rating'].hist(bins=100);

In [ ]:
listings_clean['property_type'].value_counts()
# everything samller as Other will be included in Other

In [ ]:
types = ['Boat', 'Villa', 'Entire Floor', 'Dorm', 'Camper/RV', 'Guesthouse']
for ptype in types:
    listings_clean['property_type'] = np.where(listings_clean['property_type'] == ptype, 
                                               'Other', listings_clean['property_type'])

In [ ]:
listings_clean['property_type'].value_counts()

In [ ]:
values = [7, 8, 9, 10, 11, 12, 14]
for x in values:
    listings_clean['guests_included'] = np.where(listings_clean['guests_included'] == x, 
                                                 7, listings_clean['guests_included'])

In [ ]:
listings_clean['guests_included'].value_counts()

### Resources
- https://matplotlib.org/gallery/api/two_scales.html
- https://plotly.com/python/mapbox-layers/
- https://medium.com/analytics-vidhya/plotly-for-geomaps-bb75d1de189f
- https://towardsdatascience.com/how-to-create-a-plotly-visualization-and-embed-it-on-websites-517c1a78568b
- https://towardsdatascience.com/8-clutch-ways-to-impute-missing-data-690481c6cb2b
- https://stackoverflow.com/questions/36413314/filling-missing-data-by-random-choosing-from-non-missing-values-in-pandas-datafr